<a href="https://colab.research.google.com/github/87tana/YOLO11n-Bone-Fracture-Detection-Model/blob/main/basemodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Environment Configuration

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

# Navigate to the project directory
%cd '/content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/'

Mounted at /content/drive/
/content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection


In [2]:
# Install MLflow and YOLOv8 dependencies
!pip install mlflow ultralytics pyngrok --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 623.0/623.0 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.0 MB/s eta 0:00:00


In [3]:
# Import YOLOv8 and other necessary modules
#import ultralytics
#from ultralytics import YOLO
import os
import random
import torch
import numpy as np
import mlflow.pyfunc
import time
from matplotlib import pyplot as plt

from PIL import Image
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

In [4]:
import ultralytics as ut
import mlflow

from pyngrok import ngrok
#from getpass import getpass
import subprocess

# Update a setting
ut.settings.update({"mlflow": True})

# Reset settings to default values
#ut.settings.reset()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
# Set MLflow tracking directory
mlflow_tracking_dir = "/content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/runs/mlflow"
# Set MLflow tracking URI to a folder in Google Drive
mlflow.set_tracking_uri(f"file://{mlflow_tracking_dir}")



In [ ]:
# Load a YOLO11model
#model = ut.YOLO("yolo11n.pt")  # Choose 'yolo11n.pt' for lightweight training

## Train

In [6]:
# Set experiment name
mlflow.set_experiment("Bone_Fracture_Detection")


dataset_path="/content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset"


params={'experiment_name': 'Fracture_Detection',
        'model': 'yolo11n',
        'epochs': 5,
        'img_size': 640,
        'batch_size': 2}


# Start an MLflow run

with mlflow.start_run(run_name="YOLO_Fracture_Detection") as run:
    # Log parameters
    mlflow.log_param("model", params['model'])
    mlflow.log_param("epochs", params['epochs'])
    mlflow.log_param("img_size", params['img_size'])
    mlflow.log_param("batch_size", params['batch_size'])

    # Start timer
    start_time = time.time()

    # Train the YOLO model
    model = ut.YOLO('yolo11n.pt')  # Load YOLO Nano model
    results = model.train(
        data=os.path.join(dataset_path, 'data_sample.yaml'),  # Path to your data.yaml file
        epochs=params['epochs'],  # Number of epochs
        imgsz=params['img_size'],  # Image size
        batch=params['batch_size'],  # Batch size
        #name=params['experiment_name'],  # Experiment name
        project="/content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints"  # Results directory
    )


        # Log custom metrics
    mlflow.log_metric('mAP', results.maps[0])


    # Save model weights and config as artifacts
    #mlflow.log_artifact('yolo_weights.h5')
    #mlflow.log_artifact('yolo_config.cfg')

    # Log training duration
    training_duration = time.time() - start_time
    mlflow.log_metric("training_time", training_duration)




Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/data_sample.yaml, epochs=5, time=None, patience=100, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=No

100%|██████████| 755k/755k [00:00<00:00, 81.9MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/sample_data/labels.cache... 2 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2/2 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

        5/5         0G      3.306       5.13      2.927          3        640: 100%|██████████| 1/1 [00:01<00:00,  1.46s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.63it/s]

                   all          2          2    0.00334        0.5    0.00276   0.000829



5 epochs completed in 0.008 hours.
Optimizer stripped from /content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/train11/weights/last.pt, 5.5MB
Optimizer stripped from /content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/train11/weights/best.pt, 5.5MB

Validating /content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/train11/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.92it/s]


                   all          2          2    0.00356        0.5    0.00283   0.000848
              Fracture          2          2    0.00356        0.5    0.00283   0.000848
Speed: 2.4ms preprocess, 223.4ms inference, 0.0ms loss, 17.7ms postprocess per image
Results saved to /content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/train11
MLflow: results logged to file:///content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'


In [10]:
# run tracking UI in the background
subprocess.Popen(["mlflow", "ui", "--backend-store-uri", mlflow_tracking_dir])
#get_ipython().system_raw("mlflow ui --port 5000 &") # run tracking UI in the background

NGROK_AUTH_TOKEN = "2qGXY4i9cF4CQHjBRFLtxjuQCTz_Z9PruXmBce21Hfpju7AY"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://0797-35-245-55-86.ngrok-free.app


## Validation / Test

In [ ]:
# Customize validation settings
validation_results = model.val(data="/content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/data.yaml", imgsz=640, batch=16, conf=0.25, iou=0.6, device="0")

Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


val: Scanning /content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/valid/labels.cache... 835 images, 5 backgrounds, 0 corrupt: 100%|██████████| 835/835 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 53/53 [00:19<00:00,  2.78it/s]


                   all        835       1373      0.047     0.0371     0.0248     0.0168
                person        187        202      0.094     0.0743     0.0495     0.0336
               bicycle        643       1171          0          0          0          0
Speed: 1.2ms preprocess, 5.3ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/val2


## Inference on example images and visualize the results.

In [ ]:
image_list = os.listdir('/content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/test/images')

In [11]:
# Run inference on 'bus.jpg'
results = model(os.path.join("/content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/test/images/",image_list[13]))  # results list

# Visualize the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    #im_rgb = im_rgb.resize((100, 100))

    #im_rgb

    # Show results to screen (in supported environments)
    r.show()


NameError: name 'image_list' is not defined

# Model Loading and Training

In [ ]:
#NGROK_AUTH_TOKEN = getpass('Enter the ngrok authtoken: ')
#ngrok.set_auth_token(NGROK_AUTH_TOKEN)

NGROK_AUTH_TOKEN = "2qGXY4i9cF4CQHjBRFLtxjuQCTz_Z9PruXmBce21Hfpju7AY"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

# Terminate open tunnels if exist
#ngrok.kill()



MLflow Tracking UI: https://21b9-34-16-187-179.ngrok-free.app


# Dataset Paths Definition

In [12]:
# Define dataset paths
dataset_path = '/content/drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset'

train_images_dir = os.path.join(dataset_path, 'train/images')
train_labels_dir = os.path.join(dataset_path, 'train/labels')

validation_images_dir = os.path.join(dataset_path, 'valid/images')
validation_labels_dir = os.path.join(dataset_path, 'valid/labels')

test_images_dir = os.path.join(dataset_path, 'test/images')
test_labels_dir = os.path.join(dataset_path, 'test/labels')
